In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.utilities import SQLDatabase

In [ ]:
db = SQLDatabase.from_uri("sqlite:///./transcriptions.db")

In [ ]:
def get_schema(_):
    return db.get_table_info()

In [ ]:
def run_query(query):
    return db.run(query)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

model = ChatOpenAI()

sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )

In [62]:
sql_response.invoke({"question": "based on summary given on summary column if its empty recheck summary and change the issue_category column with either Router Installation , Change / Add Device ,Product Enquiry ,TV Connection ,SMS Code Issue ,Payment Issue"})

"UPDATE transcriptions\nSET issue_category = \n    CASE \n        WHEN summary = '' THEN \n            CASE \n                WHEN transcription LIKE '%subscription activation%' THEN 'Product Enquiry'\n                WHEN transcription LIKE '%internet connection%' THEN 'Router Installation'\n                WHEN transcription LIKE '%log in%' THEN 'Change / Add Device'\n                WHEN transcription LIKE '%internet package%' THEN 'Product Enquiry'\n                ELSE 'Other'\n            END\n        ELSE issue_category\n    END"

In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
)

In [ ]:
ans = full_chain.invoke({"question": "based on summary given on summary column if its empty recheck summary and change the issue_category column with either Router Installation , Change / Add Device ,Product Enquiry ,TV Connection ,SMS Code Issue ,Payment Issue"})
import pprint
from pprint import pprint
pprint(ans)

In [ ]:
ans = full_chain.invoke({"question": "do we have angry customers?"})
import pprint
from pprint import pprint
pprint(ans)

In [ ]:
ans = full_chain.invoke({"question": "how many?"})
import pprint
from pprint import pprint
pprint(ans)

In [ ]:
#Lets Create a function out of the process
def sqlhandler(sqlfield):
    see = sql_response.invoke({"question": sqlfield})
    print(see)
    template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
    {schema}

    Question: {question}
    SQL Query: {query}
    SQL Response: {response}"""
    prompt_response = ChatPromptTemplate.from_template(template)

    full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
    )



In [ ]:
sqlfield = "based on summary given on summary column if its NULL recheck summary and change the issue_category column with either Router Installation , Change / Add Device ,Product Enquiry ,TV Connection ,SMS Code Issue ,Payment Issue" #Which table or column do you want to focus on
sqlhandler(sqlfield)
#Ask questions based on the mentioned field
ans = full_chain.invoke({"question": "based on summary given on summary column if its NULL recheck summary and change the issue_category column with either Router Installation , Change / Add Device ,Product Enquiry ,TV Connection ,SMS Code Issue ,Payment Issue   execute the query"})
import pprint
from pprint import pprint
pprint(ans)